In [2]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
from torch.utils.data import ConcatDataset
from PIL import Image
import os
import torchvision.models as models
import time
import copy
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

from dataloading import load_data


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4680, 0.4647, 0.3441], std=[0.2322, 0.2272, 0.2394]) 
])   

noise_type = "gaussian_noise"
noise_percentage = 10
data_percentage = 100


train_loader, _, classes = load_data(data_dir = '../data/vegetable_images',
                           batch_size = 64,
                           data_type = "train",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)

valid_loader, _, _ = load_data(data_dir = '../data/vegetable_images',
                           batch_size = 64,
                           data_type = "validation",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)

valid_loader_with_noise, _, _ = load_data(data_dir = '../data/vegetable_images',
                           batch_size = 64,
                           data_type = "validation",
                           noise_type = noise_type,
                           noise_percentage = noise_percentage,                           
                           transform = transform,                           
                           data_percentage=data_percentage)
dataloaders = {'train':  train_loader, 
               'val': valid_loader
               }
dataloaders_with_noise = {'train':  train_loader, 
               'val': valid_loader_with_noise
               }


test_loader, _, _ = load_data(data_dir = '../data/vegetable_images',
                           batch_size = 64,
                           data_type = "test",
                           noise_type = "gaussian_noise",
                           noise_percentage = noise_percentage,                           
                           transform = transform,                           
                           data_percentage=data_percentage)


test_loader_without_noise, _, _ = load_data(data_dir = '../data/vegetable_images',
                           batch_size = 64,
                           data_type = "test",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)





path:  ../data/vegetable_images/train
needed_length: 15000, expected_length_per_class: 1000
length of final dataset: 15000
path:  ../data/vegetable_images/validation
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000
path:  ../data/vegetable_images/gaussian_noise/validation/10
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000
path:  ../data/vegetable_images/gaussian_noise/test/10
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000
path:  ../data/vegetable_images/test
needed_length: 3000, expected_length_per_class: 200
length of final dataset: 3000


In [50]:
print(len(dataloaders['train'].dataset))
print(len(dataloaders['val'].dataset))

dataset_sizes = {'train' : len(dataloaders['train'].dataset), 'val': len(dataloaders['val'].dataset), 'test': len(test_loader.dataset)}
print(len(test_loader.dataset))
classes

15000
3000
3000


['Bean',
 'Bitter_Gourd',
 'Bottle_Gourd',
 'Brinjal',
 'Broccoli',
 'Cabbage',
 'Capsicum',
 'Carrot',
 'Cauliflower',
 'Cucumber',
 'Papaya',
 'Potato',
 'Pumpkin',
 'Radish',
 'Tomato']

In [51]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [52]:
model = models.alexnet(pretrained=True)

In [53]:
print(model._modules['classifier'])

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=4096, out_features=4096, bias=True)
  (5): ReLU(inplace=True)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)


In [54]:
in_features = model._modules['classifier'][-1].in_features
out_features = len(classes)
model._modules['classifier'][-1] = nn.Linear(in_features, out_features, bias=True)
print(model._modules['classifier'])

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=4096, out_features=4096, bias=True)
  (5): ReLU(inplace=True)
  (6): Linear(in_features=4096, out_features=15, bias=True)
)


In [55]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

cpu


In [56]:
len(test_loader_without_noise.dataset)

3000

# CLEAN IMAGES WITHOUT NOISE BEFORE TRAINING

In [6]:
alexnet = torch.load('../models/alexnet_without_noise.pth', map_location=torch.device('cpu'))

In [9]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(len(classes))]
    n_class_samples = [0 for i in range(len(classes))]
    for images, labels in test_loader_without_noise:
        images = images.to(device)
        labels = labels.to(device)
        outputs = alexnet(images)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(len(images)):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 98.9 %
Accuracy of Bean: 99.5 %
Accuracy of Bitter_Gourd: 99.0 %
Accuracy of Bottle_Gourd: 98.0 %
Accuracy of Brinjal: 97.0 %
Accuracy of Broccoli: 99.0 %
Accuracy of Cabbage: 99.5 %
Accuracy of Capsicum: 99.0 %
Accuracy of Carrot: 99.5 %
Accuracy of Cauliflower: 100.0 %
Accuracy of Cucumber: 96.0 %


In [10]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(len(classes))]
    n_class_samples = [0 for i in range(len(classes))]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = alexnet(images)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(len(images)):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 96.13333333333334 %
Accuracy of Bean: 96.5 %
Accuracy of Bitter_Gourd: 99.0 %
Accuracy of Bottle_Gourd: 96.0 %
Accuracy of Brinjal: 89.0 %
Accuracy of Broccoli: 99.5 %
Accuracy of Cabbage: 99.0 %
Accuracy of Capsicum: 74.5 %
Accuracy of Carrot: 99.5 %
Accuracy of Cauliflower: 98.0 %
Accuracy of Cucumber: 98.5 %


In [13]:
model = models.alexnet(pretrained=True)